In [1]:
import pandas as pd
%matplotlib inline
data=pd.read_csv("./boors/cleaned_data.csv")
data['date']=pd.to_datetime(data['date'])
date_range=pd.DataFrame(pd.date_range("6/22/2015", "1/1/2020", freq='D'),columns=['date'])
dfs=dict(tuple(data.groupby('unix')))
init=date_range
for d in dfs:
    dfs[d]=dfs[d][["date","open","high","low","close","volume"]]
    dfs[d]=dfs[d].add_prefix(str(d)+":")
    init=pd.merge(init,dfs[d],left_on='date',right_on=str(d)+":"+'date',how='outer')
date_cols = [col for col in init.columns[1:] if 'date' in col]
for c in date_cols:
    init[c]=init["date"]
date_cols = [col for col in init.columns[1:] if 'date' in col]
final_data=init.drop(date_cols,axis=1)
vol_cols = [col for col in final_data.columns[1:] if 'volume' in col]
final_data[vol_cols]=final_data[vol_cols].fillna(0)

In [2]:
others_cols = [col for col in final_data.columns[1:] if not 'volume' in col]

for column in others_cols:
    final_data[column].fillna(method="backfill",inplace=True)

In [4]:
# some columns that their last prices are null, should be filled with last valid data
others_cols = [col for col in final_data.columns[1:] if not 'volume' in col]

for column in others_cols:
    final_data[column].fillna(method="ffill",inplace=True)

In [5]:
final_data.to_csv("./main_final_data.csv")